In [2]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:3]

['emma', 'olivia', 'ava']

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [5]:
block_size = 3
X, Y = [], []

for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [6]:
def build_dataset(words):
    block_size = 5
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random 
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 5]) torch.Size([182625])
torch.Size([22655, 5]) torch.Size([22655])
torch.Size([22866, 5]) torch.Size([22866])


In [7]:
n_emb = 10
n_hidden = 200
block_size = 5
vocab_size = len(itos.keys())

C = torch.randn((vocab_size, n_emb))
W1 = torch.randn((n_emb*block_size, n_hidden))
b1 = torch.randn(n_hidden)
W2 = torch.randn((n_hidden, vocab_size))
b2 = torch.randn(vocab_size)


parameters = [C, W1, W2, b1, b2]
sum(p.nelement() for p in parameters)
for p in parameters:
    p.requires_grad = True

In [11]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(200000+1):
    # making batches
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Y[ix]

    # forward pass
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backwards pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    if i % 10000 == 0:
        print(f"{i}/{max_steps} Loss: {loss.item()}")

print(loss.item())

0/200000 Loss: 2.066518783569336
10000/200000 Loss: 2.733618974685669
20000/200000 Loss: 2.49685001373291
30000/200000 Loss: 2.8964920043945312
40000/200000 Loss: 3.1380443572998047
50000/200000 Loss: 2.569984197616577
60000/200000 Loss: 2.161069869995117
70000/200000 Loss: 2.5881195068359375
80000/200000 Loss: 2.3002099990844727
90000/200000 Loss: 2.4994571208953857
100000/200000 Loss: 2.224322557449341
110000/200000 Loss: 1.9578876495361328
120000/200000 Loss: 2.3972485065460205
130000/200000 Loss: 1.8960981369018555
140000/200000 Loss: 1.9641159772872925
150000/200000 Loss: 2.3219029903411865
160000/200000 Loss: 1.9940471649169922
170000/200000 Loss: 2.412080764770508
180000/200000 Loss: 2.1426286697387695
190000/200000 Loss: 2.1844823360443115
200000/200000 Loss: 1.9902058839797974
1.9902058839797974


In [12]:
@torch.no_grad
def split_loss(split):
    x, y = {
        'train' : (Xtr, Ytr),
        'val' : (Xdev, Ydev),
        'test' : (Xte, Yte)
    }[split]

    emb = C[x]
    out = emb.view(emb.shape[0], -1) @ W1 + b1
    h = torch.tanh(out)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')


train 2.1396102905273438
val 2.174565076828003
